In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
code = """
#include <stdio.h>
#include <stdlib.h>

#define N 1024
#define BLOCK_SIZE 16

__global__ void matrixMul(int *a, int *b, int *c, int width) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int sum = 0;
    for (int i = 0; i < width; i++) {
        sum += a[row * width + i] * b[i * width + col];
    }
    c[row * width + col] = sum;
}

int main() {
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int size = N * N * sizeof(int);

    // Allocate memory on host
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize matrices
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            a[i * N + j] = i + j;
            b[i * N + j] = i - j;
        }
    }

    // Allocate memory on device
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data from host to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch kernel with 2D grid and 2D block
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 dimGrid((N + dimBlock.x - 1) / dimBlock.x, (N + dimBlock.y - 1) / dimBlock.y);
    matrixMul<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, N);

    // Copy result from device to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Print first and last elements of result
    printf("c[0][0] = %d, c[%d][%d] = %d", c[0], N-1, N-1, c[(N-1) * N + (N-1)]);

    // Free memory
    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


"""

In [ ]:
text_file = open("assign4b.cu","w")
text_file.write(code)
text_file.close()

In [ ]:
!nvcc assign4b.cu

In [ ]:
!./a.out

c[0][0] = 357389824, c[1023][1023] = -714255872

In [ ]:
!nvprof ./a.out

==891== NVPROF is profiling process 891, command: ./a.out
c[0][0] = 357389824, c[1023][1023] = -714255872==891== Profiling application: ./a.out
==891== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   73.70%  9.1147ms         1  9.1147ms  9.1147ms  9.1147ms  matrixMul(int*, int*, int*, int)
                   13.17%  1.6295ms         1  1.6295ms  1.6295ms  1.6295ms  [CUDA memcpy DtoH]
                   13.13%  1.6238ms         2  811.88us  758.48us  865.29us  [CUDA memcpy HtoD]
      API calls:   89.78%  131.02ms         3  43.675ms  69.597us  130.88ms  cudaMalloc
                    9.69%  14.145ms         3  4.7150ms  1.0006ms  12.100ms  cudaMemcpy
                    0.39%  572.17us         3  190.72us  163.87us  204.74us  cudaFree
                    0.08%  121.06us       101  1.1980us     139ns  51.982us  cuDeviceGetAttribute
                    0.02%  33.479us         1  33.479us  33.479us  33.479us  cudaLaunc